In [3]:
import numpy as np 
import pandas as pd 
import tensorflow as tf


In [4]:
df = pd.read_csv("investments_VC.csv", encoding="ISO-8859-1") 
df.head()
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54294 entries, 0 to 54293
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   permalink             49438 non-null  object 
 1   name                  49437 non-null  object 
 2   homepage_url          45989 non-null  object 
 3   category_list         45477 non-null  object 
 4    market               45470 non-null  object 
 5    funding_total_usd    49438 non-null  object 
 6   status                48124 non-null  object 
 7   country_code          44165 non-null  object 
 8   state_code            30161 non-null  object 
 9   region                44165 non-null  object 
 10  city                  43322 non-null  object 
 11  funding_rounds        49438 non-null  float64
 12  founded_at            38554 non-null  object 
 13  founded_month         38482 non-null  object 
 14  founded_quarter       38482 non-null  object 
 15  founded_year       

In [5]:
df=df.dropna()
print(len(df))

21840


In [6]:
df[' funding_total_usd '] = df[' funding_total_usd '].replace({',': '', ' ': ''}, regex=True)
df[' funding_total_usd '] = df[' funding_total_usd '].apply(lambda x: x if x != '-' else '0')
df[' funding_total_usd '] = pd.to_numeric(df[' funding_total_usd '], errors='coerce')
df[' funding_total_usd '] = df[' funding_total_usd '].astype(int)

In [7]:
df.columns

Index(['permalink', 'name', 'homepage_url', 'category_list', ' market ',
       ' funding_total_usd ', 'status', 'country_code', 'state_code', 'region',
       'city', 'funding_rounds', 'founded_at', 'founded_month',
       'founded_quarter', 'founded_year', 'first_funding_at',
       'last_funding_at', 'seed', 'venture', 'equity_crowdfunding',
       'undisclosed', 'convertible_note', 'debt_financing', 'angel', 'grant',
       'private_equity', 'post_ipo_equity', 'post_ipo_debt',
       'secondary_market', 'product_crowdfunding', 'round_A', 'round_B',
       'round_C', 'round_D', 'round_E', 'round_F', 'round_G', 'round_H'],
      dtype='object')

In [8]:
df.select_dtypes('object').columns

Index(['permalink', 'name', 'homepage_url', 'category_list', ' market ',
       'status', 'country_code', 'state_code', 'region', 'city', 'founded_at',
       'founded_month', 'founded_quarter', 'first_funding_at',
       'last_funding_at'],
      dtype='object')

In [9]:
df.isnull().sum()

permalink               0
name                    0
homepage_url            0
category_list           0
 market                 0
 funding_total_usd      0
status                  0
country_code            0
state_code              0
region                  0
city                    0
funding_rounds          0
founded_at              0
founded_month           0
founded_quarter         0
founded_year            0
first_funding_at        0
last_funding_at         0
seed                    0
venture                 0
equity_crowdfunding     0
undisclosed             0
convertible_note        0
debt_financing          0
angel                   0
grant                   0
private_equity          0
post_ipo_equity         0
post_ipo_debt           0
secondary_market        0
product_crowdfunding    0
round_A                 0
round_B                 0
round_C                 0
round_D                 0
round_E                 0
round_F                 0
round_G                 0
round_H     

In [ ]:
df['first_funded_at'] = pd.to_datetime(df['first_funded_at'])
df['last_funded_at'] = pd.to_datetime(df['last_funded_at'])
df['funded_duration'] = (df['last_funded_at'] - df['first_funded_at']).dt.days / 365.25
df['years_since_funded'] = 2025 - df['first_funded_at'].dt.year

In [341]:
df.drop(columns=["permalink", "homepage_url",'country_code','state_code','region','city','founded_at', 'founded_month',
'founded_quarter','first_funding_at','last_funding_at','founded_year'], inplace=True, errors="ignore")


In [342]:
df['status'].value_counts()

status
operating    18448
acquired      2202
closed        1190
Name: count, dtype: int64

In [344]:
np.unique(df['status'])

array(['acquired', 'closed', 'operating'], dtype=object)

In [345]:
df["status"] = df["status"].apply(lambda x: 1 if x in ['acquired','operating'] else 0)
df = pd.get_dummies(df, columns=[" market ", "category_list"], drop_first=True)
df['status'].value_counts()

status
1    20650
0     1190
Name: count, dtype: int64

In [346]:
from sklearn.utils import resample
df_majority = df[df['status'] == 1]  
df_minority = df[df['status'] == 0]  

df_majority_downsampled = resample(df_majority, 
                                replace=False, 
                                n_samples=len(df_minority),
                                random_state=42) 
df_balanced = pd.concat([df_majority_downsampled, df_minority])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
print(df_balanced['status'].value_counts())
df=df_balanced

status
0    1190
1    1190
Name: count, dtype: int64


In [347]:
df.columns

Index(['name', ' funding_total_usd ', 'status', 'funding_rounds',
       'founded_year', 'seed', 'venture', 'equity_crowdfunding', 'undisclosed',
       'convertible_note',
       ...
       'category_list_|iPhone|Wireless|Android|Maps|Consumer Electronics|Gps|Reviews and Recommendations|Journalism|Video Streaming|Content|Parenting|Audio|Location Based Services|Social Media|Mobile|',
       'category_list_|iPhone|Wireless|Audio|Email|SMS|Twitter Applications|Facebook Applications|Social Network Media|Software|Databases|Mobile|',
       'category_list_|iPhone|iOS|Cloud Data Services|Startups|Technology|Mobile|',
       'category_list_|iPhone|iPod Touch|Crowdsourcing|Local|Mobile|',
       'category_list_|iPhone|iPod Touch|Mobile|Marketplaces|Recycling|E-Commerce|Hardware + Software|',
       'category_list_|iPod Touch|iPad|iPhone|Android|App Marketing|App Discovery|Mobile|',
       'category_list_|iPod Touch|iPad|iPhone|Wireless|Hardware + Software|',
       'category_list_|iPod Touch|i

In [348]:
df.head()

,name,funding_total_usd,status,funding_rounds,founded_year,seed,venture,equity_crowdfunding,undisclosed,convertible_note,...,category_list_|iPhone|Wireless|Android|Maps|Consumer Electronics|Gps|Reviews and Recommendations|Journalism|Video Streaming|Content|Parenting|Audio|Location Based Services|Social Media|Mobile|,category_list_|iPhone|Wireless|Audio|Email|SMS|Twitter Applications|Facebook Applications|Social Network Media|Software|Databases|Mobile|,category_list_|iPhone|iOS|Cloud Data Services|Startups|Technology|Mobile|,category_list_|iPhone|iPod Touch|Crowdsourcing|Local|Mobile|,category_list_|iPhone|iPod Touch|Mobile|Marketplaces|Recycling|E-Commerce|Hardware + Software|,category_list_|iPod Touch|iPad|iPhone|Android|App Marketing|App Discovery|Mobile|,category_list_|iPod Touch|iPad|iPhone|Wireless|Hardware + Software|,category_list_|iPod Touch|iPhone|iPad|Entertainment|Peer-to-Peer|Consumer Electronics|Online Shopping|Video Games|Hardware|Marketplaces|Media|Retail|E-Commerce|,category_list_|mHealth|Enterprise Software|,category_list_|mHealth|Health Care|
0,CrowdSling,40000,0,1.0,2011.0,40000.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,Supercool School,100000,0,1.0,2009.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,RampedMedia,0,0,1.0,2006.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,Quick Hit,18950000,0,5.0,2008.0,0.0,15950000.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,Boston Out-Patient Surigal Suites,160886,1,1.0,2003.0,0.0,160886.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [349]:
df.select_dtypes('object').columns

Index(['name'], dtype='object')

In [350]:
df.drop(columns=["name"],inplace=True)

In [351]:
df.head()

,funding_total_usd,status,funding_rounds,founded_year,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,...,category_list_|iPhone|Wireless|Android|Maps|Consumer Electronics|Gps|Reviews and Recommendations|Journalism|Video Streaming|Content|Parenting|Audio|Location Based Services|Social Media|Mobile|,category_list_|iPhone|Wireless|Audio|Email|SMS|Twitter Applications|Facebook Applications|Social Network Media|Software|Databases|Mobile|,category_list_|iPhone|iOS|Cloud Data Services|Startups|Technology|Mobile|,category_list_|iPhone|iPod Touch|Crowdsourcing|Local|Mobile|,category_list_|iPhone|iPod Touch|Mobile|Marketplaces|Recycling|E-Commerce|Hardware + Software|,category_list_|iPod Touch|iPad|iPhone|Android|App Marketing|App Discovery|Mobile|,category_list_|iPod Touch|iPad|iPhone|Wireless|Hardware + Software|,category_list_|iPod Touch|iPhone|iPad|Entertainment|Peer-to-Peer|Consumer Electronics|Online Shopping|Video Games|Hardware|Marketplaces|Media|Retail|E-Commerce|,category_list_|mHealth|Enterprise Software|,category_list_|mHealth|Health Care|
0,40000,0,1.0,2011.0,40000.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,100000,0,1.0,2009.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,0,0,1.0,2006.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,18950000,0,5.0,2008.0,0.0,15950000.0,0.0,0.0,0.0,3000000.0,...,False,False,False,False,False,False,False,False,False,False
4,160886,1,1.0,2003.0,0.0,160886.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [352]:
from sklearn.preprocessing import OrdinalEncoder
import json
encoder=OrdinalEncoder()
cat_col=df.select_dtypes('object').columns
print(cat_col)
cat_col
df[cat_col]=encoder.fit_transform(df[cat_col])
df[cat_col]
category_mappings = {cat_col[i]: dict(zip(encoder.categories_[i], range(len(encoder.categories_[i])))) 
                    for i in range(len(encoder.categories_))}

with open('ordinal_mappings.json', 'w') as json_file:
    json.dump(category_mappings, json_file, indent=4)

Index([], dtype='object')


In [353]:
from sklearn.preprocessing import StandardScaler
import joblib
sc=StandardScaler()
col=[' funding_total_usd ','seed','venture','debt_financing']
df[col]=sc.fit_transform(df[col])
joblib.dump(sc,"scaler.pkl")

['scaler.pkl']

In [354]:
x = df.drop(columns=['status']) 
y = df['status'] 

In [355]:
y.value_counts()

status
0    1190
1    1190
Name: count, dtype: int64

In [356]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [357]:
import tensorflow as tf
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=256,activation='relu', input_shape=(x.shape[1],)))
model.add(tf.keras.layers.Dense(units=128,activation='relu'))
model.add(tf.keras.layers.Dense(units=64,activation='relu'))
model.add(tf.keras.layers.Dense(units=32,activation='relu'))
model.add(tf.keras.layers.Dense(units=16,activation='relu'))
model.add(tf.keras.layers.Dense(units=8,activation='relu'))
model.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))


d:\Users\conne\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(x,y,epochs=150,batch_size=16)

In [ ]:
import joblib
joblib.dump(model,"startup_analyser_model.pkl")